In [182]:
all_text = []


In [183]:
same_name_event = []

In [184]:
import requests
import os

apikey ='apikey=urifgNDL2DGaCwxY1R0oBH9wGG9vbxcH'
s1 = 200
size = 'size='
title = 'https://app.ticketmaster.com/discovery/v2/events'
l_json = '.json?'
# boston 235 new york 345 Los Angeles 324
dmaId= 'dmaId=324'
# &page=4
url_1 = title + l_json + dmaId + '&' + apikey + '&' + size + str(s1) + '&page=4'
print(url_1)

r_1 = requests.get(url_1)
print('code', r_1.status_code)

response = r_1.json()
print(len(response))
print(response.keys())
tmp = response['_embedded']

events = tmp['events']
print(len(events))
print(events[0].keys())

https://app.ticketmaster.com/discovery/v2/events.json?dmaId=324&apikey=urifgNDL2DGaCwxY1R0oBH9wGG9vbxcH&size=200&page=4
code 200
3
dict_keys(['_embedded', '_links', 'page'])
200
dict_keys(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales', 'dates', 'classifications', 'promoter', 'promoters', 'info', 'pleaseNote', 'priceRanges', 'products', 'seatmap', 'ticketLimit', 'ageRestrictions', '_links', '_embedded'])


In [148]:
# a = response['errors']
# print(a)

In [185]:
# refresh 2 list !!!

this_dma_event = []
count = 0
for i_e in events:
    count += 1
#     print(i_e.keys())

    # Art_Event
    event_id = i_e['id']
    event_title = i_e['name']
    if event_title not in same_name_event:
        same_name_event.append(event_title)
    else:
        continue
    #   type  event Arts & Theatre
    event_type = i_e['type']
    subtype = i_e['classifications'][0]['segment']['name']
    if subtype in ['Music', 'Arts & Theatre']:
        event_subtype = subtype
    else:
        continue
        
    if 'seatmap' in i_e.keys():
        seatmap = i_e['seatmap']['staticUrl']
#         print(seatmap)
    else:
        continue
#     print('type', event_type, event_subtype)  
    for j in i_e['images']:
#         'width': 640, 'height': 427,
        if j['width']==640 and j['height']==427:
            event_image_url = j['url']
            break

    # Time
    time_serial = 'lz' + str(count) + '-' + event_id
    event_time = i_e['dates']['start']['localDate']
    
    # location
    location_id = i_e['_embedded']['venues'][0]['id']+'-'+ str(count)
    l_longtitude = i_e['_embedded']['venues'][0]['location']['longitude']
    l_latitude = i_e['_embedded']['venues'][0]['location']['latitude']
    # state city street
    if ',' in i_e['_embedded']['venues'][0]['address']['line1']:
        l_address = i_e['_embedded']['venues'][0]['state']['name']  + '/' + i_e['_embedded']['venues'][0]['city']['name'] + '/' + i_e['_embedded']['venues'][0]['address']['line1'].split(',')[0] + '/time_zone:' + i_e['_embedded']['venues'][0]['timezone'] 
    else:
        l_address = i_e['_embedded']['venues'][0]['state']['name']  + '/' + i_e['_embedded']['venues'][0]['city']['name'] + '/' + i_e['_embedded']['venues'][0]['address']['line1'] + '/time_zone:' + i_e['_embedded']['venues'][0]['timezone'] 
    zipcode = i_e['_embedded']['venues'][0]['postalCode']

    
    # Artist
    if 'attractions' in i_e['_embedded'].keys():
#         print(i_e['_embedded']['attractions'][0].keys())   
        artist_id = i_e['_embedded']['attractions'][0]['id'] + 'la' + str(count)
        artist_name = i_e['_embedded']['attractions'][0]['name']
        if 'externalLinks' in i_e['_embedded']['attractions'][0].keys(): 
            dic = []
#             print(i_e['_embedded']['attractions'][0]['externalLinks'].keys())
            dic.extend(i_e['_embedded']['attractions'][0]['externalLinks'].keys())
#             print(dic)
#             print(i_e['_embedded']['attractions'][0]['externalLinks'][dic[0]][0]['url'])
            if 'url' in i_e['_embedded']['attractions'][0]['externalLinks'][dic[0]][0].keys():
                artist_info = i_e['_embedded']['attractions'][0]['externalLinks'][dic[0]][0]['url']
            else:
                artist_info = i_e['_embedded']['attractions'][0]['externalLinks'][dic[0]][0]
    #         artist_info = i_e['_embedded']['attractions'][0]['externalLinks']['wiki'][0]['url'] + ',' + i_e['_embedded']['attractions'][0]['externalLinks']['facebook'][0]['url']
#         artist_info = i_e['_embedded']['attractions'][0]['externalLinks']['facebook'][0]['url']
        else:
            artist_info = 'Let google for it together'
    else:
        continue

        
    # Ticket_has
    Tid = 'T' + str(count) + '-' + event_id
    if 'priceRanges' in i_e.keys():
        price = i_e['priceRanges'][0]['min']
    else:
        price = '100'
    amount = zipcode
    refund_policy = 'yes'
#     print(i_e['sales'])
    

    if 'genre' in i_e['classifications'][0].keys():    
        e_genre = 'genre:' + i_e['classifications'][0]['genre']['name'] + '|subgenre:' + i_e['classifications'][0]['subGenre']['name']
    else:
        continue
#     print(e_genre)


    data = {'Eid': event_id, 'title': event_title, 'e_image': event_image_url, 'seatmap': seatmap, 'Aid': artist_id, 
            'artist_name': artist_name, 'info': artist_info, 'Lid': location_id, 'longitude': l_longtitude,
            'latitude': l_latitude, 'address': l_address, 'zipcode': zipcode, 'Tid': Tid, 'price': price, 
            'amount': amount, 'refund_policy': refund_policy, 'time_serial': time_serial, 
            'date_YMD': event_time, 'genre': e_genre, 'event_type': event_subtype}
#     print(data)
    this_dma_event.append(data)
    
all_text.extend(this_dma_event)
print(len(all_text))

129


In [186]:
print(all_text[0])

{'Eid': 'vv16aZAr6AaZA86Fee', 'title': '89.9 KCRW Presents The Flaming Lips', 'e_image': 'https://s1.ticketm.net/dam/a/94b/3a6818c2-17bf-47e7-b6b4-76ed6d2be94b_780911_RETINA_PORTRAIT_3_2.jpg', 'seatmap': 'https://maps.ticketmaster.com/maps/geometry/3/event/09005787149F6711/staticImage?type=png&systemId=HOST', 'Aid': 'K8vZ91716g7la1', 'artist_name': 'The Flaming Lips', 'info': 'https://www.youtube.com/user/flaminglips', 'Lid': 'KovZpZAEAl6A-1', 'longitude': '-118.30879207', 'latitude': '34.06141494', 'address': 'California/Los Angeles/3790 Wilshire Blvd./time_zone:America/Los_Angeles', 'zipcode': '90010', 'Tid': 'T1-vv16aZAr6AaZA86Fee', 'price': 47.5, 'amount': '90010', 'refund_policy': 'yes', 'time_serial': 'lz1-vv16aZAr6AaZA86Fee', 'date_YMD': '2020-06-17', 'genre': 'genre:Rock|subgenre:Alternative Rock', 'event_type': 'Music'}


In [187]:
import json
# los_data
jsonfile = open('/Users/yby/Documents/2020spring/542project/los/la_data.json','a')
json.dump(all_text, jsonfile,ensure_ascii=False)
jsonfile.close()

In [188]:
link = '/Users/yby/Documents/2020spring/542project/los/'

In [189]:
import json
import re

def first_process(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 

In [190]:
event_type_list = ['Music', 'Arts & Theatre' ]
# with open('test.csv', 'a') as w:
with open(link + 'art_event.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
#     with open('Los_Angeles.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,title,e_image,seatmap' + '\n'
        w.write(first_line)
        for i in s:
            if i['event_type'] in event_type_list:
                t_text = first_process(i['title'])
#             for words in t_text.split(' '):
#                 event_name_1 = 
#             line = i['Eid'] + ',' + i['title'] + ',' + i['e_image'] + '\n'
                line = i['Eid'] + ',' + t_text + ',' + i['e_image'] + ',' + i['seatmap'] + '\n'
#             print(len(i['e_image']))
                w.write(line)

In [191]:
with open(link + 'location.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Lid,longitude,latitude,address,zipcode' + '\n'
        w.write(first_line)
        for i in s:
            t_text = first_process(i['title'])
            line = i['Lid'] + ',' + i['longitude'] + ',' + i['latitude'] + ',' + i['address'] + ',' + i['zipcode'] + '\n'
            w.write(line)

In [192]:
with open(link + 'held.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,Lid' + '\n'
        w.write(first_line)
        for i in s:
#             t_text = first_process(i['title'])
            line = i['Eid'] + ',' + i['Lid'] + '\n'
            w.write(line)

In [193]:
with open(link + 'time.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'time_serial,date_YMD' + '\n'
        w.write(first_line)
        for i in s:
#             t_text = first_process(i['title'])
            line = i['time_serial'] + ',' + i['date_YMD'] + '\n'
            w.write(line)

In [194]:
with open(link + 'T_on.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,time_serial' + '\n'
        w.write(first_line)
        for i in s:
#             t_text = first_process(i['title'])
            line = i['Eid'] + ',' + i['time_serial'] + '\n'
            w.write(line)

In [195]:

with open(link + 'Ticket_has.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Tid,Eid,price,amount,refund_policy' + '\n'
        w.write(first_line)
        for i in s:
#             t_text = first_process(i['title'])
            line = i['Tid'] + ',' + i['Eid'] + ',' + str(i['price']) + ',' + str(i['amount']) + ',' + i['refund_policy'] + '\n'
            w.write(line)

In [196]:
event_type_list = ['Music']

with open(link + 'concert.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,concert_type' + '\n'
        w.write(first_line)
        for i in s:
            if i['event_type'] in event_type_list:
                line = i['Eid'] + ',' + i['genre'] + '\n'
                w.write(line)

In [197]:
event_type_list = ['Arts & Theatre']

with open(link + 'Theater.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,genre' + '\n'
        w.write(first_line)
        for i in s:
            if i['event_type'] in event_type_list:
                line = i['Eid'] + ',' + i['genre'] + '\n'
                w.write(line)

In [198]:
same_a = []
with open(link + 'Artist.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Aid,artist_name,info' + '\n'
        w.write(first_line)
        for i in s:
            if i['Aid'] not in same_a:
                same_a.append(i['Aid'])
#             t_text = first_process(i['title'])
                line = i['Aid'] + ',' + i['artist_name'] + ',' + str(i['info'])  + '\n'
                w.write(line)
            else:
                continue

In [199]:
same_a = []
with open(link + 'Perform.csv', 'a') as w:
    with open(link + 'la_data.json','r',encoding='utf-8') as r:
        s = json.load(r)
        first_line = 'Eid,Aid' + '\n'
        w.write(first_line)
        for i in s:
            if i['Aid'] not in same_a:
                same_a.append(i['Aid'])
                line = i['Eid'] + ',' + i['Aid'] + '\n'
                w.write(line)
            else:
                continue

In [139]:
s = []
with open('/Users/yby/Documents/2020spring/542project/los/Artist.csv','r',encoding='utf-8') as r2:
    for i in r2:
        s.append(i)
s1=[]
with open('/Users/yby/Documents/2020spring/542project/bos/Artist.csv','r',encoding='utf-8') as r3:
    for j in r3:
        s1.append(j)
        
s2=[]
with open('/Users/yby/Documents/2020spring/542project/ny/new_artist.csv','r',encoding='utf-8') as r4:
    for k in r4:
        s2.append(k)


In [141]:
ret1 = list(set(s1).difference(set(s)))


In [142]:
 ret2 = list(set(ret1).difference(set(s2)))

In [144]:
with open('/Users/yby/Documents/2020spring/542project/bos/new_artist.csv','a',encoding='utf-8') as w:
        first_line = 'Aid,artist_name,info' + '\n'
        w.write(first_line)
        for i in ret2:
            w.write(i)